In [1]:
import numpy as np
import time
#from ipyexperiments import *
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import os
import gc
import torch 
import pandas as pd
import time
import cProfile
from sklearn.metrics import precision_score,recall_score,roc_auc_score,confusion_matrix


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.set_default_tensor_type('torch.cuda.FloatTensor')
image_size=128  
num_epoch=5


In [3]:
input_path="/home/harsh/amk/128*128_Resized_imgs_of_good_bad/"

In [4]:
model = models.resnet34(pretrained=True).cuda()
model.fc = nn.Sequential(nn.Linear(512, 256),nn.ReLU(inplace=True),nn.Linear(256,128),nn.ReLU(inplace=True),nn.Linear(128,2)).cuda()

In [36]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [37]:
batchsize=128            

In [38]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
           

In [39]:
data_transforms = {
                    'train':
                    transforms.Compose([
                        transforms.Resize((image_size,image_size)),
                        transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
                        transforms.RandomHorizontalFlip(),
                        transforms.ToTensor(),
                        normalize
                    ]),
                    'val':
                    transforms.Compose([
                        transforms.Resize((image_size,image_size)),
                        transforms.ToTensor(),
                        normalize
                    ]),
                }

           

In [40]:
image_datasets = {
                    'train': 
                    datasets.ImageFolder(input_path + 'train', data_transforms['train']),
                    'val': 
                    datasets.ImageFolder(input_path + 'val', data_transforms['val'])
                }

           


In [41]:
 dataloaders = {'train':torch.utils.data.DataLoader(image_datasets['train'],batch_size=batchsize,shuffle=True,num_workers=0,),
                 'val':
                    torch.utils.data.DataLoader(image_datasets['val'],
                                                batch_size=batchsize,
                                                shuffle=True,
                                                num_workers=0,)
                                            #sampler=ImbalancedDatasetSampler(image_datasets['validation']))  
                }


In [42]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters())

In [43]:
accuracy=[]
predictions=[]
targets=[]
for epoch in range(num_epoch):
    epoch_preds=[]
    epoch_labels=[]
    print('Epoch {}/{}'.format(epoch+1, num_epoch))
    print('-' * 10)

    for phase in ['train']:
        since=time.time()
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:
            inputs = inputs.cuda()
            labels = labels.cuda()

            outputs = model(inputs)
            outputs=outputs.cuda()
            loss = criterion(outputs, labels)

            if phase == 'train':
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            preds=list(map(int,preds))
            labels=list(map(int,labels.data.cpu()))
            epoch_preds.extend(preds)
            epoch_labels.extend(labels)
            predictions.extend(preds)
            targets.extend(labels)
        time_taken=(time.time()-since)/float(60)
        epoch_loss = running_loss / len(image_datasets[phase])
        epoch_acc = running_corrects.double() / len(image_datasets[phase])
        precision=precision_score(epoch_labels,epoch_preds,average='macro')
        recall=recall_score(epoch_labels,epoch_preds,average='macro')
        #roc_score=roc_auc_score(epoch_labels,epoch_preds,average='micro')
        print(f'Precision::{precision} Recall::{recall}')

        print('{} loss: {:.4f}, acc: {:.4f}, time: {}'.format(phase,epoch_loss,epoch_acc,time_taken))


cnf_matrix=confusion_matrix(targets, predictions)


torch.cuda.empty_cache()
gc.collect()

torch.save(model.state_dict(),f'./Normal_Abnormal_{image_size}_.pth')



            

Epoch 1/5
----------


RuntimeError: CUDA out of memory. Tried to allocate 32.00 MiB (GPU 0; 5.80 GiB total capacity; 3.28 GiB already allocated; 24.88 MiB free; 50.05 MiB cached)

In [13]:
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### ONXX

In [14]:
from torch.autograd import Variable
import io
import numpy as np
from torch import nn
import torch.utils.model_zoo as model_zoo
import torch.onnx
import torchvision
import torch

In [15]:
# dummy_input = Variable(torch.randn(1, 3, 512, 512))
# state_dict = torch.load('Normal_Abnormal_64_.pth')
# model.load_state_dict(state_dict)
# torch.onnx.export(model, dummy_input, "normal_abnormal_clf.onnx",export_params=True)

In [16]:
# Input to the model
x = torch.randn(1, 3, 128, 128, requires_grad=True)
torch_out = model(x)

# Export the model
torch.onnx.export(model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "norm_abnorm.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  )

In [ ]:
import onnx

onnx_model = onnx.load("norm_abnorm.onnx")
onnx.checker.check_model(onnx_model)

In [1]:
import onnxruntime

ort_session = onnxruntime.InferenceSession("norm_abnorm.onnx")

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

NameError: name 'x' is not defined

In [ ]:
from PIL import Image
import torchvision.transforms as transforms

img = Image.open("img_256.png")

In [131]:
to_tensor = transforms.ToTensor()
img_y = to_tensor(img)
img_y.unsqueeze_(0)

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]], device='cpu')

In [132]:
img_y.shape

torch.Size([1, 3, 256, 256])

In [133]:
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(img_y)}
ort_outs = ort_session.run(None, ort_inputs)
img_out_y = ort_outs[0]

In [134]:
img_out_y

array([[-5.159274 ,  4.4847274]], dtype=float32)